In [1]:
import os
import numpy as np
import torch
import glob
from PIL import Image
import cv2
import manga109api
import torchvision
from torchvision import transforms
from src.dataset import Resizer, Normalizer
from src.model import EfficientDet
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

In [2]:
PATH = "mangadet.pth"

In [ ]:
model = EfficientDet(num_classes = 2).cuda()
model.load_state_dict(torch.load(PATH))
model.eval()

In [ ]:
image_base_path = "F:\\Datasets\\Manga109\\images\\"
manga109_root_dir = "F:\\Datasets\\Manga109"
p = manga109api.Parser(root_dir=manga109_root_dir)

In [ ]:
book = "ARMS"
image_full_path = image_base_path + book + '\\'
annotation = p.get_annotation(book=book)

In [ ]:
page_index = 19
img = cv2.imread(p.img_path(book=book, index=page_index))
#img = cv2.imread("5.jpg")
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = cv2.resize(img, (512,512))
img = img.astype(np.float32) / 255.
img = torch.from_numpy(img).unsqueeze(0)
img = img.permute(0, 3, 1, 2).double().cuda()

In [ ]:
with torch.no_grad():
    out = model(img)

In [ ]:
img = cv2.imread(p.img_path(book=book, index=page_index))
#img = cv2.imread("5.jpg")
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = cv2.resize(img, (512,512))
for co in out[2]:
    color = (255, 0, 0)
    thickness = 2
    start_point = (int(co[0]), int(co[1]))
    end_point = (int(co[2]), int(co[3]))
    img = cv2.rectangle(img, start_point, end_point, color, thickness)
plt.figure(figsize=(10,12)) 
plt.imshow(img)

In [ ]:
cv2.imwri